Help method for reading and formating data

In [54]:
import pandas as pd

def read_and_format(filePath, columns_to_remove):
    df = pd.read_csv(filePath, delimiter=',', low_memory=False)
    
    # removing unnecessary columns
    df = df.drop(columns_to_remove, axis=1)
    # renaming columns
    df.rename(columns={'location-long':'longitude'}, inplace = True) 
    df.rename(columns={'location-lat':'latitude'}, inplace = True)    
    # adding species columns
    df.insert(0, 'species', 'griffon_vulture', True)

    return df

Help method for removing coordinates that are near each other (removes only other coordinate)

In [55]:
import numpy as np
from math import ceil
from scipy.spatial import distance
from haversine import haversine, Unit

def remove_near_coordinates(df, min_distance=50, count=1000):
    print(len(df.index))
    
    iteration_count = ceil(len(df.index) / count)
    idx_out = list()

    for i in range(iteration_count):
        lower_bound = i * count
        mini_df = df.iloc[lower_bound:lower_bound + count,1:3]
        coords = mini_df.values

        distance_matrix = distance.cdist(coords, coords, lambda u, v: haversine(u, v, Unit.METERS)) # haversine(lat, lon, Unit.Meters)
        n_proxim = np.apply_along_axis(lambda x: np.count_nonzero(x < min_distance), 0, distance_matrix)
        idx = np.argsort(n_proxim).tolist()

        for i in idx:
            for j in range(i+1):
                if i not in idx_out:
                    if distance_matrix[i, j] < min_distance:
                        if i != j:
                            idx_out.append(j)

    pop_idx = sorted(np.unique(idx_out).tolist(), reverse=True)
    df = df.drop(df.index[pop_idx])

    print(len(df.index))
    return df

In [56]:
def clean(df):
    # removing NaN values
    df = df.dropna()    
    # removing duplicates
    df = df.drop_duplicates()
    # removing coordinates if distance between two is below given min_distance
    #df = remove_near_coordinates(df)
    return df

Help method for writing cleaned data to final dataset file

In [57]:
def to_csv(filePath, df):
    df.to_csv(filePath, mode='a', index=False, header=False)

Main

In [58]:
import os

datasets_info = [
    ('../MovebankDatasets/e-ObsGPRSHimalayanGriffon-Bhutan-MPIAB', ['event-id', 'visible', 'timestamp', 'bar:barometric-pressure','data-decoding-software','eobs:activity','eobs:activity-samples','eobs:battery-voltage','eobs:fix-battery-voltage','eobs:horizontal-accuracy-estimate','eobs:key-bin-checksum','eobs:speed-accuracy-estimate','eobs:start-timestamp','eobs:status','eobs:temperature','eobs:type-of-fix','eobs:used-time-to-get-fix','gps:dop','gps:satellite-count','ground-speed','heading','height-above-ellipsoid','import-marked-outlier','mag:magnetic-field-raw-x','mag:magnetic-field-raw-y','mag:magnetic-field-raw-z','orientation:quaternion-raw-w','orientation:quaternion-raw-x','orientation:quaternion-raw-y','orientation:quaternion-raw-z','sensor-type','individual-taxon-canonical-name','tag-local-identifier','individual-local-identifier','study-name','utm-easting','utm-northing','utm-zone','study-timezone','study-local-timestamp']),
    ('../MovebankDatasets/Eurasian Griffon Vultures 1 Hz HUJ (Israel)',['event-id','visible','timestamp','bar:barometric-pressure','data-decoding-software','eobs:activity','eobs:activity-samples','eobs:battery-voltage','eobs:fix-battery-voltage','eobs:horizontal-accuracy-estimate','eobs:key-bin-checksum','eobs:speed-accuracy-estimate','eobs:start-timestamp','eobs:status','eobs:temperature','eobs:type-of-fix','eobs:used-time-to-get-fix','gps:dop','gps:satellite-count','ground-speed','heading','height-above-ellipsoid','height-raw','import-marked-outlier','mag:magnetic-field-raw-x','mag:magnetic-field-raw-y','mag:magnetic-field-raw-z','orientation:quaternion-raw-w','orientation:quaternion-raw-x','orientation:quaternion-raw-y','orientation:quaternion-raw-z','taxon','sensor-type','individual-taxon-canonical-name','tag-local-identifier','individual-local-identifier','study-name','utm-easting','utm-northing','utm-zone','study-timezone','study-local-timestamp']),
    ('../MovebankDatasets/Griffon vulture [fdlmes.gr]',['event-id','visible','timestamp','sensor-type','individual-taxon-canonical-name','tag-local-identifier','individual-local-identifier','study-name','utm-easting','utm-northing','utm-zone','study-timezone','study-local-timestamp']),
    ('../MovebankDatasets/Griffon Vulture Albstadt Salzburg (Gypsi)', ['event-id','visible','timestamp','bar:barometric-pressure','data-decoding-software','eobs:activity','eobs:activity-samples','eobs:battery-voltage','eobs:fix-battery-voltage','eobs:horizontal-accuracy-estimate','eobs:key-bin-checksum','eobs:speed-accuracy-estimate','eobs:start-timestamp','eobs:status','eobs:temperature','eobs:type-of-fix','eobs:used-time-to-get-fix','gps:dop','gps:satellite-count','ground-speed','heading','height-above-ellipsoid','import-marked-outlier','mag:magnetic-field-raw-x','mag:magnetic-field-raw-y','mag:magnetic-field-raw-z','manually-marked-outlier','orientation:quaternion-raw-w','orientation:quaternion-raw-x','orientation:quaternion-raw-y','orientation:quaternion-raw-z','sensor-type','individual-taxon-canonical-name','tag-local-identifier','individual-local-identifier','study-name','utm-easting','utm-northing','utm-zone','study-timezone','study-local-timestamp']),
    ('../MovebankDatasets/High-altitude flights of Himalayan vultures (data from Sherub et al. 2016)', ['event-id','visible','timestamp','eobs:battery-voltage','eobs:fix-battery-voltage','eobs:horizontal-accuracy-estimate','eobs:speed-accuracy-estimate','eobs:start-timestamp','eobs:status','eobs:temperature','eobs:type-of-fix','eobs:used-time-to-get-fix','ground-speed','heading','height-above-ellipsoid','sensor-type','individual-taxon-canonical-name','tag-local-identifier','individual-local-identifier','study-name','utm-easting','utm-northing','utm-zone','study-timezone','study-local-timestamp']),
    ('../MovebankDatasets/LifeTrack Griffon Vulture Croatia', ['event-id','visible','timestamp','bar:barometric-pressure','data-decoding-software','eobs:activity','eobs:activity-samples','eobs:battery-voltage','eobs:fix-battery-voltage','eobs:horizontal-accuracy-estimate','eobs:key-bin-checksum','eobs:speed-accuracy-estimate','eobs:start-timestamp','eobs:status','eobs:temperature','eobs:type-of-fix','eobs:used-time-to-get-fix','gps:dop','gps:satellite-count','ground-speed','heading','height-above-ellipsoid','import-marked-outlier','mag:magnetic-field-raw-x','mag:magnetic-field-raw-y','mag:magnetic-field-raw-z','orientation:quaternion-raw-w','orientation:quaternion-raw-x','orientation:quaternion-raw-y','orientation:quaternion-raw-z','sensor-type','individual-taxon-canonical-name','tag-local-identifier','individual-local-identifier','study-name','utm-easting','utm-northing','utm-zone','study-timezone','study-local-timestamp']),
    ('../MovebankDatasets/Long-range adult movements of 3 vulture species (data from Spiegel et al. 2015)', ['event-id','visible','timestamp','ground-speed','heading','sensor-type','individual-taxon-canonical-name','tag-local-identifier','individual-local-identifier','study-name','utm-easting','utm-northing','utm-zone','study-timezone','study-local-timestamp']),
    ('../MovebankDatasets/Raptors NABU Moessingen public', ['event-id','visible','timestamp','bar:barometric-pressure','data-decoding-software','eobs:activity','eobs:activity-samples','eobs:battery-voltage','eobs:fix-battery-voltage','eobs:horizontal-accuracy-estimate','eobs:key-bin-checksum','eobs:speed-accuracy-estimate','eobs:start-timestamp','eobs:status','eobs:temperature','eobs:type-of-fix','eobs:used-time-to-get-fix','gps:dop','gps:satellite-count','ground-speed','heading','height-above-ellipsoid','import-marked-outlier','mag:magnetic-field-raw-x','mag:magnetic-field-raw-y','mag:magnetic-field-raw-z','orientation:quaternion-raw-w','orientation:quaternion-raw-x','orientation:quaternion-raw-y','orientation:quaternion-raw-z','sensor-type','individual-taxon-canonical-name','tag-local-identifier','individual-local-identifier','study-name','utm-easting','utm-northing','utm-zone','study-timezone','study-local-timestamp']),
    ('../MovebankDatasets/Soaring flight in Eurasian griffon vultures (HUJ) (data from Harel and Nathan, 2018)', ['event-id','visible','timestamp','ground-speed','heading','height-raw','sensor-type','individual-taxon-canonical-name','tag-local-identifier','individual-local-identifier','study-name','utm-easting','utm-northing','utm-zone','study-timezone','study-local-timestamp'])
]
final_dataset_file_path = '../FinalDatasets/griffon_vulture.csv'

for dataset_info in datasets_info: 
    file_path = dataset_info[0]
    dataset_files = os.listdir(file_path)

    for file in dataset_files:
        df = read_and_format(file_path + '/' + file, dataset_info[1])
        df = clean(df)
        
        to_csv(final_dataset_file_path, df)

In [62]:
# clean final dataset
df = pd.read_csv(final_dataset_file_path, delimiter=',')
print(len(df.index))
df = df.drop_duplicates()
print(len(df.index))

os.remove(final_dataset_file_path)
to_csv(final_dataset_file_path, df)

2251416
2251416
